# Getting Data Asynchronously

Let me share some horrors upon horrors with you. Let's build a function to handle asychronous calls to the server... only imagine it's pre-2005:

-----

```javascript
function ajax(url, data, method, async, contentType, dataType, success, failure, always) {
            function _parseResult(request) {
                switch (dataType.lower()) {
                    case "text":
                        return request.responseText;
                    case "json":
                        return request.responseJSON();
                    case "xml":
                        return <XMLDocument>request.responseXML;
                    default:
                        return request.responseText;
                }
            }
            function _send(request, data) {
                var ajx = this;
                request.open(method, url, async, data);
                request.onreadystatechange = function () {
                    if (request.readyState === 4) {
                        if (request.status === 200) {
                            if (success !== undefined) {
                                success(_parseResult(request));
                            }
                        }
                        if (request.status === 404 || request.status === 405 || request.status === 500) {
                            if (failure !== undefined) {
                                failure(request.responseText, request.responseJSON(), request.responseXML);
                            }
                            else {
                                //...Generic error
                            }
                        }
                        if (always !== undefined) {
                            always(request);
                        }
                    }
                };
                request.setRequestHeader("Content-Type", contentType);
                if (url.contains("localhost")) {
                    request.setRequestHeader("Cache-Control", "max-age=0");
                }
                request.send(data);
            }
            let request = new XMLHttpRequest();
            let requestData = (typeof(data) !== "string") ? querystringify(data, true) : data;
            if (requestData.startsWith("?")) {
                requestData = requestData.substr(1);
            }
            if (requestData.endsWith("?")) {
                requestData = requestData.substr(0, requestData.length - 2);
            }
            var self = {
                url: url
                , method: method
                , contentType: contentType
                , dataType: dataType
                , data: requestData
                , async: async
                , request: request
                , send: function () {
                    _send(request, requestData);
                }
            };
            if (success != null || failure != null || always != null) {
                self.send();
            }
            return self;
        }
```

> Note: Not all code to execute this is present. Yes... there's even more.

## jQuery kind of saved us here too

Instead of managing AJAX calls by hand, you could use several different convenience libraries to help you.

### The very simpliest?

```javascript
    $.ajax(); //Uses a global settings object to infer additional data.
```

### The standard?

You'll see most jQuery calls with some form of URL, data parsing, and what to do upon completion:

```javascript
$.ajax({
    method: "POST",
    url: "example.aspx",
    data: { UserID: "michael@szul.us" }
})
.done(function(res){
    alert("Hey, you posted some data!");
});
```

jQuery will handle the above code like a typical JavaScript `Promise` with the `done()` function acting as the `thennable` call.

## But why use jQuery? Or any framework for that matter?

Modern JavaScript gives us the **Fetch API** to simplify vanilla AJAX calls.

```javascript
fetch("example.aspx")
    .then((res) => {
        alert("Hey, you acquired some data!");
    })
    .catch((err) => {
        console.log(err);
    });
```

In the above example, we connect to the endpoint, which returns a `Promise` or throws an error. We can access the results in the `then()` method.

If we wanted to post data like the jQuery example:

```javascript
var opts = {
    method: "POST",
    headers: {
        "Content-Type": "application/json"
    },
    body: JSON.stringify({ UserID: "michael@szul.us" })
};
fetch("example.aspx", opts)
    .then((res) => {
        alert("Hey, you posted some data!");
    })
    .catch((err) => {
        console.log(err);
    });
```

It's a little more verbose than the jQuery example, but it allows more customization, and doesn't have a need for external libraries.

### Async/Await

The Fetch API looks a lot cleaner when you take async/await into account too:

```javascript
var jwt = JSON_WEB_TOKEN;
var response = await fetch(`api/EPA/count?UserID=${computingID}`, {
    mode: "cors",
    credentials: "include",
    headers: {
        Authorization: `Bearer ${jwt}`
    }
});
var result = await response.json();
```

## The power of the Fetch API with Service Workers

The Fetch API is even more powerful when combined with new JavaScript events meant to work with Service Workers. For example, you could intercept fetch requests in order to serve data from cache.

```javascript
self.addEventListener('fetch', function (event) {
    var request = event.request;
    event.respondWith(
      caches.match(request).then((response) => {
          if (response) {
              return response;
          }
          else {
              return fetch(request).then((res) => {
                  return res;
              }).catch((err) => {
                  console.error(`Error: Failed to cache: ${err}`);
              });
          }
      })
    );
});
```
